<a href="https://colab.research.google.com/github/amitesh554/Data-Visualization-on-titanic/blob/main/ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
df=pd.read_csv("train.csv")
df_test=pd.read_csv("test.csv")



In [4]:
df["Age"].fillna(df["Age"].median(),inplace=True)
df["Embarked"].fillna("S",inplace=True)
df["Fare"].fillna(df["Fare"].median(),inplace=True)

In [5]:
def get_title(name):
    if "." in name:
        return name.split(',')[1].split('.')[0].strip()
    else:
        return "No title in name"

title=sorted(set([x for x in df.Name.map(lambda x:get_title(x))]))


def shorten_title(x):
    title=x["Title"]
    if title in ['Capt','Col','Major']:
        return "Officer"
    elif title in ['Jonkheer','Don','the Countess','Dr','Lady','Sir']:
        return "Royalty"
    elif title in 'Mme':
        return 'Mrs'
    elif title in ['Mlle','Ms']:
        return "Miss"
    else:
        return title

df['Title']=df['Name'].map(lambda x:get_title(x))
df['Title']=df.apply(shorten_title,axis=1)

In [6]:
df.drop("Name",axis=1,inplace=True)
df.drop("Ticket",axis=1,inplace=True)
df.drop("Cabin",axis=1,inplace=True)
df.Sex.replace(('male','female'),(0,1),inplace=True)
df.Embarked.replace(('S','C','Q'),(0,1,2),inplace=True)
df.Title.replace(('Mr','Miss','Mrs','Master','Royalty','Rev','Officer'),(0,1,2,3,4,5,6),inplace=True)


In [7]:
cor=df.corr()
cor.Survived.sort_values(ascending=False)


Survived       1.000000
Sex            0.543351
Title          0.377715
Fare           0.257307
Embarked       0.106811
Parch          0.081629
PassengerId   -0.005007
SibSp         -0.035322
Age           -0.064910
Pclass        -0.338481
Name: Survived, dtype: float64

In [8]:
from sklearn.model_selection import train_test_split

x=df.drop(['Survived','PassengerId'],axis=1)
y=df['Survived']

x_train,x_val,y_train,y_val=train_test_split(x,y,test_size=0.1)

In [9]:
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

randomforest=RandomForestClassifier()
randomforest.fit(x_train,y_train)

y_pred=randomforest.predict(x_val)

acc_randomfor=round(accuracy_score(y_pred,y_val)*100,2)

print(acc_randomfor)

pickle.dump(randomforest,open('titanic_model.sav','wb'))

81.11


In [10]:
df_test=pd.read_csv("test.csv")
df_test['Title']=df_test['Name'].map(lambda x:get_title(x))
df_test['Title']=df_test.apply(shorten_title,axis=1)

ids=df_test['PassengerId']

df_test['Age'].fillna(df_test['Age'].median(),inplace=True)
df_test['Embarked'].fillna("S",inplace=True)
df_test['Fare'].fillna(df_test['Fare'].median(),inplace=True)

df_test.drop("Name",axis=1,inplace=True)
df_test.drop("Ticket",axis=1,inplace=True)
df_test.drop("Cabin",axis=1,inplace=True)
df_test.drop("PassengerId",axis=1,inplace=True)

df_test.Sex.replace(('male','female'),(0,1),inplace=True)
df_test.Embarked.replace(('S','C','Q'),(0,1,2),inplace=True)
df_test.Title.replace(('Mr','Miss','Mrs','Master','Royalty','Rev','Officer','Dona'),(0,1,2,3,4,5,6,7),inplace=True)


In [11]:
predictions=randomforest.predict(df_test)
output=pd.DataFrame({"PassengerId":ids,"Survived":predictions})
output.to_csv("Submission.csv",index=False)

